In [ ]:
from pathlib import Path
from typing import List, Iterable
import pandas as pd

In [ ]:
# データや出力先のフォルダを指定
DATA_DIR = Path("data")
OUTPUT_DIR = Path("poc_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# データファイルの名前パターン
OPERATION_PATTERN = "★機器遠隔操作履歴_*.csv"
STATUS_PATTERN = "★機器状態変化履歴_*.csv"

# 後に、NULLに置き換え
NA_VALUES = ["", "NULL", '""（空文字）', "（空文字）"]

# 分析対象者を設定
CONTRACT_IDS = [
    # 対象者の契約者IDを入力
]

# 機器遠隔操作履歴から抽出したいカラム
OPERATION_COLUMNS = [
    "ContractId",
    "OrderReceiptDate", # -> タイムスタンプ
    "TimerDiv",         # -> 操作区分 (タイマー種別)"
    "FloorCode",
    "RoomName",
    "EquipmentTypeId",
    "EquipmentName",
    "PropertyCode",     # -> 実行コード
    "PropertyName",     # -> 実行コードの日本語説明
    "PropertyValue",    # -> 実行コードのパラメータ
]

# 機器状態変化履歴から抽出したいカラム
STATUS_COLUMNS = [
    "MessageName",      # -> 通知種別 (生きているか)
    "ContractId",       
    "ReportedDate",     # -> タイムスタンプ
    "FloorCode",
    "RoomName",
    "EquipmentTypeId",
    "EquipmentName",
    "PropertyCode1",
    "PropertyName1",
    "PropertyValue1",
]

In [ ]:
def select_columns(df: pd.DataFrame, columns: Iterable[str]) -> pd.DataFrame:
    return df[[col for col in columns if col in df.columns]]


def load_filtered(pattern: str, columns: Iterable[str]) -> pd.DataFrame:
    frames: List[pd.DataFrame] = []
    for csv_path in sorted(DATA_DIR.glob(pattern)):
        df = pd.read_csv(csv_path, encoding="utf-8-sig", na_values=NA_VALUES)
        filtered = df[df["ContractId"].isin(CONTRACT_IDS)]
        if filtered.empty:
            continue
        filtered = select_columns(filtered, columns)
        frames.append(filtered)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=columns)


operation_df = load_filtered(OPERATION_PATTERN, OPERATION_COLUMNS)
status_df = load_filtered(STATUS_PATTERN, STATUS_COLUMNS)

# 確認_機器遠隔操作履歴
print("operation_df:")
display(operation_df.head())

# 確認_機器状態変化履歴
print("status_df:")
display(status_df .head())

In [ ]:
### 機器遠隔操作履歴
# カラム名を変更し、連結準備
operation_df = operation_df.rename(columns={
    "OrderReceiptDate": "ts",
    "PropertyCode": "property_code",
    "PropertyName": "property_name",
    "PropertyValue": "property_value",
})

# 遠隔操作であることのフラグを立てる
operation_df["is_remote"] = True

###

### 機器状態変化履歴
# カラム名を変更し、結合準備
status_df = status_df.rename(columns={
    "ReportedDate": "ts",
    "PropertyCode1": "property_code",
    "PropertyName1": "property_name",
    "PropertyValue1": "property_value",
})

# 生活者が意識的に機器状態を変化させた場合に限定
status_df = status_df[status_df["MessageName"]=="equipmentStatusIndication"]

# 遠隔操作であることのフラグを立てない
status_df["is_remote"] = False

# 機器遠隔操作履歴にしかないカラムを NaN にしておく
status_df["TimerDiv"] = np.nan

###

# 日時を datetime に変換（タイムゾーン付き文字列もパース可能）
operation_df["ts"] = pd.to_datetime(operation_df["ts"], errors="coerce")
status_df["ts"] = pd.to_datetime(status_df["ts"], errors="coerce")

# 連結後のカラムを定義
common_cols = [
    "ts",
    "ContractId",
    "is_remote",
    "TimerDiv",
    "EquipmentTypeId",
    "EquipmentName",
    "property_code",
    "property_name",
    "property_value",
]

# 不要なカラムを削除
operation_df = operation_df[common_cols]
status_df = status_df[common_cols]

# 連結
logs_df = pd.concat([operation_df, status_df], ignore_index=True)

# 並べ替え（契約・時間順）
logs_df = logs_df.sort_values(
    ["ContractId", "ts"]
).reset_index(drop=True)

# 確認
logs_df.head(50)